In [84]:
from statsbombpy import sb
import pandas as pd
from datetime import datetime
from meteostat import Point, Daily
from tqdm import tqdm

In [75]:
stadiums = pd.read_csv("data/stadiums-with-GPS-coordinates.csv")
matches = pd.read_csv("data/premier-league-matches.csv")

# Renaming Columns so it is the same as in the stadiums dataset
matches.rename(columns={"Home":"Team"}, inplace = True)
# Trimming whitespaces in the stadium dataset
stadiums = stadiums.applymap(lambda x: x[:-1] if type(x) == str and x[-1] == " " else x)
# Contains coordinates and date for each game
matches = matches.merge(stadiums, on="Team")

In [83]:
data.columns

Index(['tavg', 'tmin', 'tmax', 'prcp', 'snow', 'wdir', 'wspd', 'wpgt', 'pres',
       'tsun'],
      dtype='object')

In [91]:
# Now we're gonna use the meteostat library to retrieve weather data for each match

output_file = "data/weather.csv"
columns = [
    'stadium', 
    'date', 
    'tavg', 
    'tmin', 
    'tmax', 
    'prcp', 
    'snow', 
    'wdir', 
    'wspd', 
    'wpgt', 
    'pres',
    'tsun']

# Creating header
with open(output_file, 'w') as f:
    f.write(columns[0])
    for col in columns[1:]:
        f.write(",")
        f.write(col)
    f.write("\n")
    
for i in tqdm(range(len(matches))):
    stadium = matches["Stadium"][i]
    latitude = float(matches["Latitude"][i])
    longitude = float(matches["Longitude"][i])
    date_str = matches["Date"][i]
    year, month, day = map(int, date_str.split("-"))
    
    # API Call
    start = datetime(year, month, day)
    end   = datetime(year, month, day)
    location = Point(latitude, longitude, 70)
    data = Daily(location, start, end)
    data = data.fetch()
    
    # Data not found
    if data.empty:
        continue
    # Writing data
    with open(output_file, 'a') as f:
        f.write(stadium)
        f.write(",")
        f.write(date_str)
        for col in columns[2:]:
            f.write(",")
            f.write(str(data[col][0]))
        f.write("\n")

100%|███████████████████████████████████████| 7541/7541 [03:06<00:00, 40.34it/s]


In [98]:
weather_df = pd.read_csv("data/weather.csv")
matches_with_weather = matches.merge(weather_df, left_on = ["Stadium","Date"], right_on = ["stadium","date"], copy=True)

In [ ]:
matches_with_weather.